In [17]:
from tensorflow.keras import models, layers, utils
import matplotlib.pyplot as pyplot
import tensorflow as tf

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model

In [18]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data() 


In [32]:
model = models.Sequential()

# Pierwsza warstwa konwolucyjna
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

# Druga warstwa konwolucyjna
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# trzecia warstwa konwolucyjna
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))


# czwarta warstwa konwolucyjna
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

# Zamiana trójwymiarowej macierzy na jednowymiarowy wektor (przestawienie kolejnych wierszy kolumn liniowo)
model.add(layers.Flatten())

# Sieć neuronowa
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 64)         

In [20]:
x_train[0].shape

(32, 32, 3)

In [33]:

model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:



x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255




In [48]:
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 98s 2ms/sample - loss: 0.6933 - acc: 0.7593 - val_loss: 0.7240 - val_acc: 0.7442
Epoch 2/10
50000/50000 [==============================] - 91s 2ms/sample - loss: 0.6863 - acc: 0.7622 - val_loss: 0.7498 - val_acc: 0.7391
Epoch 3/10
50000/50000 [==============================] - 91s 2ms/sample - loss: 0.6807 - acc: 0.7619 - val_loss: 0.7270 - val_acc: 0.7471
Epoch 4/10
50000/50000 [==============================] - 90s 2ms/sample - loss: 0.6819 - acc: 0.7633 - val_loss: 0.7433 - val_acc: 0.7409
Epoch 5/10
50000/50000 [==============================] - 91s 2ms/sample - loss: 0.6737 - acc: 0.7630 - val_loss: 0.7480 - val_acc: 0.7451
Epoch 6/10
50000/50000 [==============================] - 93s 2ms/sample - loss: 0.6742 - acc: 0.7629 - val_loss: 0.7274 - val_acc: 0.7494
Epoch 7/10
50000/50000 [==============================] - 91s 2ms/sample - loss: 0.6698 - acc: 0.7651 - val_loss: 0.

In [49]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7130045212745667
Test accuracy: 0.7546


In [50]:
layer_name = "flatten"
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)


intermediate_output = intermediate_layer_model.predict(x_train)

In [51]:
intermediate_output.shape

(50000, 128)

In [52]:
from sklearn import svm

clf = svm.SVC()

clf.fit(intermediate_output, y_train)

/home/tomasz/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [53]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(intermediate_layer_model.predict(x_test))

accuracy_score(y_test, y_pred)

0.7707